In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

new_employed = pd.read_csv('Data/new_employed.csv')
new_employed = new_employed.drop('Unnamed: 0',1)

new_employed['Income Level']= new_employed['Monthly Income']
#level_1 = new_employed['Income Level'].quantile(.10)
#level_2 = new_employed['Income Level'].quantile(.20)
#level_3 = new_employed['Income Level'].quantile(.30)
level_4 = new_employed['Income Level'].quantile(.40)
#level_5 = new_employed['Income Level'].quantile(.50)
#level_6 = new_employed['Income Level'].quantile(.60)
#level_7 = new_employed['Income Level'].quantile(.70)
level_8 = new_employed['Income Level'].quantile(.80)
#level_9 = new_employed['Income Level'].quantile(.90)

In [4]:
new_employed.loc[(new_employed['Monthly Income'] <= level_4, 'Income Level')] = 'Level 1'
new_employed.loc[((new_employed['Monthly Income'] <= level_8) & (new_employed['Monthly Income'] > level_4),'Income Level')] = 'Level 2'
#new_employed.loc[((new_employed['Monthly Income'] <= level_6) & (new_employed['Monthly Income'] > level_4), 'Income Level')] = 'Level 6'
new_employed.loc[(new_employed['Monthly Income'] > level_8, 'Income Level')] = 'Level 3'

new_employed['Income Level'].value_counts()

Level 1    119408
Level 3     35886
Level 2     25076
Name: Income Level, dtype: int64

In [5]:
y = new_employed['Income Level']
X = new_employed.drop(columns=['Monthly Income', 'Income Level'], axis=1)

rel_columns = ['Age','Gender', 'Literacy', 'Scholarship','Career', 'Marital Status', 'Urban or Rural', 'Residence','Economic Zone', 'By Position','By Economic Sector']
X = pd.get_dummies(X[rel_columns], drop_first=True, dtype=float)

scaler = MinMaxScaler()
X=pd.DataFrame(scaler.fit_transform(X),
            columns=X.columns, index=X.index) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= 42)

clf = DecisionTreeClassifier(criterion='entropy')

clf.fit(X_train, y_train)

y_preds = clf.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_preds))

Accuracy:  0.5882168135868863


In [6]:
# with gini

target = new_employed['Income Level']
data = new_employed.drop(columns=['Monthly Income', 'Income Level'], axis=1)

data = pd.get_dummies(data[rel_columns], drop_first=True, dtype=float)

scaler = MinMaxScaler()
data=pd.DataFrame(scaler.fit_transform(data),
            columns=data.columns, index=data.index) 

data_train, data_test, target_train, target_test = train_test_split(data, target, 
                                                                    test_size = 0.25, random_state=123)
# Instantiate and fit a DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(criterion='gini', max_depth=5) 
tree_clf.fit(data_train, target_train)

DecisionTreeClassifier(max_depth=5)

In [7]:
# Test set predictions
pred = tree_clf.predict(data_test)

# classification report

print(classification_report(target_test, pred))
print("Testing Accuracy for Decision Tree Classifier: {:.4}%".format(accuracy_score(target_test, pred) * 100))

              precision    recall  f1-score   support

     Level 1       0.68      0.97      0.80     29867
     Level 2       0.47      0.03      0.05      6240
     Level 3       0.43      0.08      0.14      8986

    accuracy                           0.67     45093
   macro avg       0.52      0.36      0.33     45093
weighted avg       0.60      0.67      0.56     45093

Testing Accuracy for Decision Tree Classifier: 66.55%


In [8]:
# Prunning

# Instantiate and fit a DecisionTreeClassifier
tree_clf2 = DecisionTreeClassifier(criterion='gini', min_samples_split=8, max_depth=100, splitter='best', min_samples_leaf= 100) 
tree_clf2.fit(data_train, target_train)

tree_clf2.score(data_train, target_train)

0.6824146011517109

In [9]:
tree_clf2.score(data_test, target_test)

0.6748497549508793

In [10]:
# Instantiate a BaggingClassifier
bagged_tree2 =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', min_samples_split=8, max_depth=100, splitter='best', min_samples_leaf= 100), 
                                 n_estimators=20)
# Fit to the training data
bagged_tree2.fit(data_train, target_train)

# Training accuracy score
bagged_tree2.score(data_train, target_train)

0.6813427264058192

In [11]:
# Test accuracy score
bagged_tree2.score(data_test, target_test)

0.6763577495398398